In [1]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np

from edgecraft import *


# font
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["font.size"] = 15
plt.rcParams["legend.fontsize"] = 15

# ticks
plt.rcParams["xtick.labelsize"] = 10
plt.rcParams["ytick.labelsize"] = 10
plt.rcParams["xtick.top"] = True
plt.rcParams["xtick.labeltop"] = False
plt.rcParams["ytick.right"] = True
plt.rcParams["ytick.labelright"] = False
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"

# figure
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams['figure.subplot.left'] = 0.15

**Simulation Parameters**

In [2]:
hbar = h / (2 * np.pi)
n = 1e15        # m^-2
B_0 = n * magnetic_flux_quantum / 1
dielectric_vacuum = 8.8854e-12      # F/m
dielectric_GaAs = 12.9 * dielectric_vacuum
m_e = 9.109e-31     # kg
m_GaAs = 0.067 * m_e
k_B = 1.380649e-23
T = 40e-3

M = 20  # multiplier of magnetic length
l_B = np.sqrt(h / (2 * np.pi * e * B_0))
l = l_B * M

E_0 = e**2 / (4 * np.pi * dielectric_GaAs * l)
E_LL_diff = hbar * e * B_0 / (m_GaAs * E_0)
E_F = E_LL_diff / 2
E_QH = E_F / 2

E_gate_min = 0
E_gate_max = E_F

U_thermal = k_B * T / E_0
U_disorder = (60e-6 * e) / E_0
U_fluc = U_disorder + U_thermal

In [3]:
print(f"E_F = {E_F}")
print(f"U_fluc = {U_fluc}")
print(f"2 um / l = {2e-6 / l}")

E_F = 8.104990760116417
U_fluc = 0.14391857894307894
2 um / l = 7.926654595212022


**Definition of Space**

In [4]:
radius_gate = int(50e-6 / l)

x = np.arange(0, int(200e-6 / l), 1)
y = np.arange(0, int(150e-6 / l), 1)
Y, X = np.meshgrid(y, x)

space_matrix = (
    true_circle_in(Y, X, y[len(y) // 2], x[len(x) // 2], radius_gate) |
    (Y >= y[len(y) // 2])
).astype(int)
diff_y = np.gradient(space_matrix, 1, axis=0)
diff_x = np.gradient(space_matrix, 1, axis=1)

# sample edge
boundary = ((diff_x != 0) | (diff_y != 0)).astype(int)
boundary_y = y[np.where(boundary == 1)[1]]
boundary_x = x[np.where(boundary == 1)[0]]
boundary_indices = np.array(np.where(boundary == 1)).T

# bulk (space \ sample edge)
bulk = np.copy(space_matrix)
bulk[boundary == 1] = 0
bulk_indices = np.array(np.where(bulk == 1)).T

# vacuum
vacuum = np.copy((~space_matrix.astype(bool)).astype(int))

# expansion gate
gate = np.logical_xor(
    true_circle_in(Y, X, y[len(y) // 2], x[len(x) // 2], radius_gate),
    true_circle_in(Y, X, y[len(y) // 2], x[len(x) // 2], radius_gate - int(25e-6 / l)),
).astype(int)
gate[Y >= y[len(y) // 2]] = 0
gate[boundary == 1] = 0
gate_indices = np.array(np.where(gate == 1)).T

**Adjusting the parameter of the confinement potential $\alpha$**

In [5]:
energy = np.zeros_like(space_matrix, dtype=float)
energy = apply_confinement_potential(energy, bulk_indices, boundary_indices, 1)

In [6]:
print(np.max(energy))

2.2769578957317074


In [7]:
alpha = 1e3

# PoC interpretation

**Execution check without applying local potential**

In [8]:
energy = np.zeros_like(space_matrix, dtype=float)
energy = apply_QH_energy(energy, E_QH, bulk_indices)
energy = apply_confinement_potential(energy, bulk_indices, boundary_indices, alpha)

In [9]:
edge = find_edge(energy, E_F, U_fluc, bulk)

In [10]:
calculate_edge_length(edge)

np.float64(1032.1164975263728)

In [ ]:
len(x) + np.pi * radius_gate - 2 * radius_gate

1018.035345410779